# Azure ML Workspace

## Importing libraries

In [1]:
from azureml.core import Workspace, Environment, Model\
    # , Dataset
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
import requests
import json

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (cryptography 3.1.1 (c:\users\jack\anaconda3\lib\site-packages), Requirement.parse('cryptography<4.0.0,>=3.3.1; extra == "crypto"'), {'PyJWT'}).


## Initialise workspace
Initialise a workspace using the values from config.json
If this file is not found, an exception occurs
When an exception is detected, a workspace using the defined details is created

In [2]:
try:
    ## Try to load workspace
    wspace = Workspace.from_config()
except Exception as ex:
    ## Create new workspace if existing is not found
    wspace = Workspace.create(name = "music-genre-classification",
                                  subscription_id = "11636c4c-f408-4202-980a-b738e97f4d7f",
                                  resource_group = "ml-classification",
                                  create_resource_group = True,
                                  location = "uksouth")

    ## Write workspace config to a local file
    wspace.write_config()

print("Workspace successfully loaded")

Deploying AppInsights with name musicgeninsights14245d5b.
Deployed AppInsights with name musicgeninsights14245d5b. Took 4.0 seconds.
Deploying KeyVault with name musicgenkeyvault4ae666d9.
Deploying StorageAccount with name musicgenstoragee08c371bd.
Deployed KeyVault with name musicgenkeyvault4ae666d9. Took 19.18 seconds.
Deploying Workspace with name music-genre-classification.
Deployed StorageAccount with name musicgenstoragee08c371bd. Took 20.74 seconds.
Deployed Workspace with name music-genre-classification. Took 19.98 seconds.
Workspace successfully loaded


## Create an environment
Create an environment for the model to run on the Azure servers

In [3]:
##  Load conda dependencies required for this environment.
##  Register environment for the workspace
env = Environment("env")

conda_dep = CondaDependencies.create(pip_packages = ["pandas == 1.1.3",
                                                    "azureml-core",
                                                    "azureml-defaults >= 1.0.45"],
                                    conda_packages = ["scikit-learn == 0.23.2"])

env.python.conda_dependencies = conda_dep

env.register(workspace = wspace)

print("Environment Registered")

## Get the environment created from the workspace and create a file to verify it has been defined
my_env = Environment.get(workspace = wspace, name = "env")
my_env.save_to_directory("./environ", overwrite = True)

print("Environment Saved")

inference_config = InferenceConfig(entry_script = "score.py",
                                   environment = my_env)

Environment Registered
Environment Saved


## Register the models
Register the saved model created by "knn-model-tuning" and "svm-model-tuning" for use on Azure ML

In [4]:
knn_model = Model.register(workspace = wspace,
                       model_name = "knn",
                       model_path = "models/knn.pkl",
                       model_framework = "ScikitLearn",
                       description = "Tuned KNN for Genre Classification")

svm_model = Model.register(workspace = wspace,
                       model_name = "svm",
                       model_path = "models/svm.pkl",
                       model_framework = "ScikitLearn",
                       description = "Tuned SVM for Genre Classification")

Registering model knn
Registering model svm


## Set up Azure Container Instance
Define ACI Webservice

In [5]:
aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1,
    memory_gb = 1)


## Deploy the models
Deploy the models onto the defined ACI container using the set environment, inference config and workspace

In [6]:
service = Model.deploy(workspace = wspace,
                       name = "music-genre-service",
                       models = [knn_model, svm_model],
                       inference_config = inference_config,
                       deployment_config = aci_config,
                       overwrite = True)

service.wait_for_deployment(show_output = True)
print(service.get_logs())

print(service.scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running....................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
2021-04-19T17:16:13,340446000+00:00 - rsyslog/run 
2021-04-19T17:16:13,341416200+00:00 - iot-server/run 
2021-04-19T17:16:13,359848500+00:00 - gunicorn/run 
2021-04-19T17:16:13,381074100+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_d322234cf1103dbd7b07da4ab58adb31/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_d322234cf1103dbd7b07da4ab58adb31/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_d322234cf1103dbd7b07da4ab58adb31/lib/libssl.so.1.0.0: no ver

## Test the web service
Test the web service has successfully deployed using test data

In [7]:
url = service.scoring_uri

# Setting strict to false allows the JSON data to be loaded without errors

data = json.load(open("test_data.json"), strict=False)

# First method uses the requests library to POST data to the web service

response = requests.post(url, json=data)

print(response.json())

# Second method uses the build in run method within the WebService class

response_run = service.run(json.dumps(data))

print(response_run)

{'KNN': ['Rock', 'Metal', 'Country', 'Classical', 'Rock', 'HipHop', 'Jazz', 'Reggae', 'Pop', 'Blues'], 'SVM': ['Reggae', 'HipHop', 'HipHop', 'Classical', 'Metal', 'HipHop', 'Blues', 'Reggae', 'Pop', 'Metal']}
{'KNN': ['Rock', 'Metal', 'Country', 'Classical', 'Rock', 'HipHop', 'Jazz', 'Reggae', 'Pop', 'Blues'], 'SVM': ['Reggae', 'HipHop', 'HipHop', 'Classical', 'Metal', 'HipHop', 'Blues', 'Reggae', 'Pop', 'Metal']}


## Delete ACI
Delete the ACI instance to clean up
(This will also delete the endpoint)

In [8]:
# service.delete()